# Proyecto 2 - ML
#### Grupo 1:  Gledson - Eva - Sara - Rubén - Luis Q

### - - Importación de módulos y librerías - - 

In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from selenium import webdriver

from Scrappers.Scrapper_Comentarios import extrae_comentarios_meneo

# Extracción de URLS
Cargamos el dataset de "Meneos" para utilizar las URL como índice sobre el cual iterar para extraer los comentarios

In [2]:
df_completo = pd.read_csv("Data/MeneosCompleto.csv", sep=";")
df_completo.shape

(274376, 12)

Eliminamos Nans y duplicados

In [3]:
df_URLs = df_completo[df_completo["URL"].isna() == False]["URL"]
df_URLs = df_URLs.drop_duplicates()
df_URLs.shape

(274364,)

# Bucle para el Scrapping de Comentarios

Es posible establecer el límite de comentarios que se quiere extraer. Decidimos extaer 100.000

In [4]:

browser = webdriver.Chrome()

limite_comentarios = 100000
contador_comentarios = 0
lista_aux_comentarios = [] 

# Recorremos el listado de enlaces para extraer los comentarios de cada noticia
for meneo in df_URLs:
    # Mientras no hayamos superado el contador, continuamos recorriendo el bucle de URLs
    if contador_comentarios < limite_comentarios:
        pagina_comentarios = 1

        # Accedemos a la URL con el sufijo "standard" para disponer de los comentarios ordenados
        browser.get(f"{meneo}/standard/")
        Soup_pagina_meneo = BeautifulSoup(browser.page_source, "html.parser")

        # Si la noticia tiene más de 100 comentarios, obtenemos el número de páginas de comentarios que tiene
        try:
            total_paginas_comentarios = int(Soup_pagina_meneo.find("div", class_="pages")["data-total-pages"])
        except:
            total_paginas_comentarios = 1

        # En función del número de páginas, las recorremos mientras haya
        while pagina_comentarios <= total_paginas_comentarios:
            if pagina_comentarios == 1:
                resultado = extrae_comentarios_meneo(Soup_pagina_meneo, pagina_comentarios)
            else:
                browser.get(f"{meneo}/standard/{pagina_comentarios}")
                Soup_pagina_meneo = BeautifulSoup(browser.page_source, "html.parser")
                resultado = extrae_comentarios_meneo(Soup_pagina_meneo, pagina_comentarios)
            
            # Almacenamos los comentarios en una lista y actualizamos el contador
            lista_aux_comentarios.extend(resultado)
            contador_comentarios += len(resultado)
            pagina_comentarios += 1
    else:
        break
    
browser.close()

In [5]:
# Almacenamos los resultado en un DataFrame
df = pd.DataFrame(lista_aux_comentarios)

In [6]:
# Exportamos el DataFrame a un CSV
df.to_csv(f'Data/{contador_comentarios}_Comentarios.csv', sep=';', encoding='utf-8', index=False)